In [39]:
from flask import Flask, jsonify
import numpy as np
import pandas as pd
import datetime as dt
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [40]:
# Set up the database
engine = create_engine("sqlite:///hawaii.sqlite")
Base = automap_base()
Base.prepare(autoload_with=engine)


In [41]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [42]:
# Create our session (link) from Python to the DB
session = Session(engine)


In [43]:
# Set up Flask
app = Flask(__name__)

In [44]:
# Define the home route
@app.route("/")
def welcome():
    return (
        f"Welcome to the Climate API!<br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start><br/>"
        f"/api/v1.0/<start>/<end><br/>"
    )


In [45]:
# Define the precipitation route
@app.route("/api/v1.0/precipitation")
def precipitation():
    # Calculate the date one year from the last date in data set.
    most_recent_date = session.query(func.max(Measurement.date)).one()[0]
    one_year_ago= dt.datetime.strptime(most_recent_date, '%Y-%m-%d') - dt.timedelta(days=365)


In [46]:
# Perform a query to retrieve the data and precipitation scores
precipitation_data = session.query(Measurement.date, Measurement.prcp)\
        .filter(Measurement.date >= one_year_ago).all()




NameError: name 'one_year_ago' is not defined

In [ ]:
# Convert the query results to a dictionary using date as the key and prcp as the value
    precipitation_dict = {date: prcp for date, prcp in precipitation_data}

    return jsonify(precipitation_dict)



In [ ]:

# Define the stations route
@app.route("/api/v1.0/stations")
def stations():
    # Perform a query to retrieve the station data
    stations_data = session.query(Station.station).all()


In [ ]:
 # Convert the query results to a list
    stations_list = [station[0] for station in stations_data]

    return jsonify(stations_list)

In [ ]:
# Define the tobs route
@app.route("/api/v1.0/tobs")
def tobs():
    # Calculate the date one year from the last date in data set.
    most_recent_date = session.query(func.max(Measurement.date)).one()[0]
    one_year_ago = dt.datetime.strptime(most_recent_date, '%Y-%m-%d') - dt.timedelta(days=365)

In [ ]:
 # Query the dates and temperature observations of the most-active station for the previous year of data.
    most_active_station = session.query(Measurement.station, func.count(Measurement.station))\
        .group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).first()[0]
    temperature_data = session.query(Measurement.date, Measurement.tobs)\
        .filter(Measurement.station == most_active_station).filter(Measurement.date >= one_year_ago).all()


In [ ]:
# Convert the query results to a list of temperature observations
    tobs_list = [temp[1] for temp in temperature_data]

    return jsonify(tobs_list)

In [ ]:
# Define the start route
@app.route("/api/v1.0/<start>")
def start(start):
    # Query the minimum, average, and maximum temperatures for all dates greater than or equal to the start date.
    temperature_stats = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs))\
        .filter(Measurement.date >= start).all()

In [ ]:
# Convert the query results to a list
    temperature_list = list(np.ravel(temperature_stats))

    return jsonify(temperature_list)


In [ ]:
# Define the start-end route
@app.route("/api/v1.0/<start>/<end>")
def start_end(start, end):
    # Query the minimum, average, and maximum temperatures for dates from the start date to the end date, inclusive.
    temperature_stats = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs))\
        .filter(Measurement.date >= start).filter(Measurement.date <= end).all()

In [ ]:
 # Convert the query results to a list
    temperature_list = list(np.ravel(temperature_stats))

    return jsonify(temperature_list)

if __name__ == "__main__":
    app.run(debug=True)